In [1]:
import numpy as np
import pandas as pd
import requests
import googlemaps
from itertools import tee
import json

In [2]:
API_key = '!!!enter your Google Maps API key here!!!' #enter Google Maps API key
gmaps = googlemaps.Client(key=API_key)

ValueError: Invalid API key provided.

In [69]:
#Read CSV file into data frame named 'df'
#change depot#
df = pd.read_excel("depot-to-consumer_depot-10.xlsx")
df.head()

,ID,longitude,latitude,city
0,9,-118.431390,34.051950,Depot-9
1,17,-118.488409,34.019890,Street-1
2,18,-118.487261,34.019897,Street-2
3,19,-118.488409,34.019890,Street-3
4,20,-118.488409,34.019890,Street-4


In [70]:
#empty list - will be used to store calculated distances
time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [72]:
for (i1, row1) in df.iterrows():
    print("origin")
    print(row1['ID'])
    LatOrigin = row1['latitude']
    LongOrigin = row1['longitude']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['ID'] 
    for (i2, row2) in  df.iterrows():
        LatDestination = row2['latitude']
        LongDestination = row2['longitude']
        destination_id = row2['ID']
        destination = (LatDestination, LongDestination)
        result = gmaps.distance_matrix(origin, destination, mode = 'driving')
        result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
        result_time = result["rows"][0]["elements"][0]["duration"]["value"]

        time_list.append(result_time)
        distance_list.append(result_distance)
        origin_id_list.append(origin_id)
        destination_id_list.append(destination_id)

origin
9


ApiError: REQUEST_DENIED (You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started)

In [74]:
output = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output['duration in seconds'] = time_list
output['origin_id'] = origin_id_list
output['destination_id'] = destination_id_list
output

,Distance in meter,duration in seconds,origin_id,destination_id
0,11000,970,249,9
1,0,0,9,9
2,11000,986,249,9
3,3284,519,249,17
4,3416,509,249,18
...,...,...,...,...
41885,5543,788,367,363
41886,5866,795,367,364
41887,2305,383,367,365
41888,1550,287,367,366


In [75]:
output_v1 = pd.merge(output, df, how = "left", left_on = "origin_id", right_on = "ID")
output_v2 = pd.merge(output_v1, df, how = "left", left_on = "destination_id", right_on = "ID")
output_v2
output_v2[['city_x', 'city_y', 'Distance in meter', 'duration in seconds']]
output_v2.to_excel('depot-10_arccs.xlsx')  #change depot#